In [ ]:
#@title 1. Thiết lập Môi trường và Kết nối Google Drive
# Import các thư viện cơ bản
import pandas as pd
import numpy as np
import os

# Kết nối với Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Đã kết nối thành công với Google Drive!")
# Cài đặt các thư viện cần thiết cho các bước sau (nếu cần)
# pandas, numpy đã có sẵn trên Colab
# !pip install unidecode # Có thể cần để xử lý chuỗi tiếng Việt sau này

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Đã kết nối thành công với Google Drive!


In [ ]:
#@title 2. Tải Dữ liệu (Categories và Products)
# --- !!! QUAN TRỌNG: THAY ĐỔI ĐƯỜNG DẪN NÀY CHO ĐÚNG VỚI DRIVE CỦA BẠN !!! ---
DRIVE_PATH = '/content/drive/MyDrive/tiki_data_crawled'
# --------------------------------------------------------------------------------

CATEGORIES_FILE = os.path.join(DRIVE_PATH, 'tiki_book_categories.csv')
PRODUCTS_FILE = os.path.join(DRIVE_PATH, 'tiki_all_book_products.csv')

try:
    # Đọc file danh mục
    df_categories = pd.read_csv(CATEGORIES_FILE)
    print(f"✅ Đã tải thành công file danh mục! Có {len(df_categories)} danh mục.")

    # Đọc file sản phẩm
    df_products = pd.read_csv(PRODUCTS_FILE)
    print(f"✅ Đã tải thành công file sản phẩm! Có {len(df_products)} sản phẩm.")

except FileNotFoundError:
    print(f"❌ LỖI: Không tìm thấy file tại đường dẫn '{DRIVE_PATH}'.")
    print("Vui lòng kiểm tra lại biến DRIVE_PATH và đảm bảo thư mục/file tồn tại trên Google Drive của bạn.")

# Hiển thị thông tin cơ bản và 5 dòng đầu của mỗi DataFrame
print("\n--- Thông tin DataFrame Danh mục ---")
df_categories.info()
print("\n--- 5 Dòng đầu của Dữ liệu Danh mục ---")
print(df_categories.head())


print("\n\n--- Thông tin DataFrame Sản phẩm ---")
df_products.info()
print("\n--- 5 Dòng đầu của Dữ liệu Sản phẩm ---")
print(df_products.head())

✅ Đã tải thành công file danh mục! Có 755 danh mục.
✅ Đã tải thành công file sản phẩm! Có 282276 sản phẩm.

--- Thông tin DataFrame Danh mục ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category_id    755 non-null    int64 
 1   category_name  755 non-null    object
 2   parent_id      755 non-null    int64 
 3   url_key        755 non-null    object
 4   level          755 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 29.6+ KB

--- 5 Dòng đầu của Dữ liệu Danh mục ---
   category_id      category_name  parent_id                      url_key  \
0          320      English Books       8322               sach-tieng-anh   
1          316    Sách tiếng Việt       8322       sach-truyen-tieng-viet   
2         7741     Văn phòng phẩm       8322  van-phong-pham-qua-luu-niem   
3        18328       Quà lưu niệm       8322      

In [ ]:
#@title 3. Làm sạch Dữ liệu Sản phẩm (Xử lý giá trị thiếu)

print("--- Bắt đầu làm sạch dữ liệu sản phẩm ---")

# Sao chép để tránh thay đổi DataFrame gốc
df_products_cleaned = df_products.copy()

# Xử lý các giá trị thiếu cho các cột quan trọng
# Điền "Không rõ" cho các trường văn bản
df_products_cleaned['author_brand_name'].fillna("Không rõ", inplace=True)
df_products_cleaned['product_name'].fillna("Không có tên", inplace=True)

# Điền 0 cho các trường số liệu
numeric_cols_to_fill_zero = [
    'price', 'original_price', 'discount_rate',
    'rating_average', 'review_count', 'quantity_sold'
]
for col in numeric_cols_to_fill_zero:
    df_products_cleaned[col].fillna(0, inplace=True)

# Chuyển đổi các cột số về đúng kiểu dữ liệu (integer/float)
df_products_cleaned['rating_average'] = pd.to_numeric(df_products_cleaned['rating_average'], errors='coerce').fillna(0).astype(float)
df_products_cleaned['quantity_sold'] = pd.to_numeric(df_products_cleaned['quantity_sold'], errors='coerce').fillna(0).astype(int)
df_products_cleaned['review_count'] = pd.to_numeric(df_products_cleaned['review_count'], errors='coerce').fillna(0).astype(int)


print("\n✅ Đã hoàn tất việc làm sạch dữ liệu cơ bản.")
print("\n--- Kiểm tra lại kiểu dữ liệu sau khi làm sạch ---")
print(df_products_cleaned[['author_brand_name', 'rating_average', 'quantity_sold']].info())

print("\n--- Dữ liệu sản phẩm sau khi làm sạch (5 dòng đầu) ---")
print(df_products_cleaned.head())

--- Bắt đầu làm sạch dữ liệu sản phẩm ---

✅ Đã hoàn tất việc làm sạch dữ liệu cơ bản.

--- Kiểm tra lại kiểu dữ liệu sau khi làm sạch ---


<ipython-input-3-dbf7c5600500>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_products_cleaned['author_brand_name'].fillna("Không rõ", inplace=True)
<ipython-input-3-dbf7c5600500>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value,

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282276 entries, 0 to 282275
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   author_brand_name  282276 non-null  object 
 1   rating_average     282276 non-null  float64
 2   quantity_sold      282276 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 6.5+ MB
None

--- Dữ liệu sản phẩm sau khi làm sạch (5 dòng đầu) ---
   product_id    product_sku  \
0      416389  4674425006297   
1    32720778  1573715963148   
2     3320515  1988630280650   
3   272532949  7668228112357   
4    22881852  5587108239013   

                                        product_name  \
0  Truyện đọc tiếng Anh - Wordsworth Editions: Th...   
1         Sách tiếng Anh - Principles: Life and Work   
2  Sách thiếu nhi tiếng Anh - Peppa Pig: Little L...   
3                                   Batman: Year One   
4                  English Grammar in Use Book w 

In [ ]:
#@title 4. Hợp nhất Dữ liệu (Join Products và Categories)

print("--- Bắt đầu hợp nhất dữ liệu sản phẩm và danh mục ---")

# Đảm bảo df_categories đã được tải ở Cell 2
if 'df_categories' not in globals():
    print("LỖI: DataFrame 'df_categories' chưa được tải. Vui lòng chạy lại Cell 2.")
else:
    # Chọn các cột cần thiết từ df_categories để join
    # và đổi tên cột 'category_name' để tránh trùng lặp nếu df_products_cleaned đã có cột tên tương tự
    df_categories_to_join = df_categories[['category_id', 'category_name', 'level']].copy()
    df_categories_to_join.rename(columns={'category_name': 'joined_category_name', 'level': 'category_level'}, inplace=True)

    # Dùng left join để giữ lại tất cả sản phẩm, join dựa trên 'crawled_category_id' của sản phẩm
    # và 'category_id' của danh mục
    df_merged = pd.merge(
        df_products_cleaned, # Sử dụng DataFrame đã được làm sạch từ Cell 3
        df_categories_to_join,
        left_on='crawled_category_id',
        right_on='category_id',
        how='left'
    )

    # Điền tên danh mục "Không rõ" cho những sản phẩm không join được (ít khả năng xảy ra nếu crawl đúng)
    df_merged['joined_category_name'].fillna("Không rõ", inplace=True)
    df_merged['category_level'].fillna(0, inplace=True) # Điền level 0 nếu không join được

    # Xóa cột category_id thừa từ df_categories_to_join sau khi join
    if 'category_id_y' in df_merged.columns: # Pandas có thể tự thêm _x, _y nếu tên cột trùng
        df_merged.drop(columns=['category_id_y'], inplace=True)
    elif 'category_id' in df_merged.columns and 'crawled_category_id' in df_merged.columns and df_merged.columns.tolist().count('category_id') > 1:
         # Trường hợp phức tạp hơn, cần xác định đúng cột để xóa
         # Tạm thời, nếu cột 'category_id' vẫn còn và khác với 'crawled_category_id', có thể đó là cột thừa
         # Cách an toàn hơn là đổi tên trước khi merge hoặc chỉ định suffixes
         pass # Sẽ kiểm tra kỹ hơn ở các cột được chọn ở cell cuối

    # Đổi tên lại cột category_id từ df_categories (nếu nó được giữ lại với tên khác)
    if 'category_id_x' in df_merged.columns:
        df_merged.rename(columns={'category_id_x': 'category_id_from_join_key'}, inplace=True)


    print(f"✅ Hợp nhất hoàn tất. Số lượng sản phẩm trong bảng mới: {len(df_merged)}")
    print(f"Các cột mới được thêm vào hoặc cập nhật: 'joined_category_name', 'category_level'")

    print("\n--- Dữ liệu sau khi hợp nhất (5 dòng đầu) ---")
    print(df_merged.head())

    print("\n--- Thông tin DataFrame sau khi hợp nhất ---")
    df_merged.info()

--- Bắt đầu hợp nhất dữ liệu sản phẩm và danh mục ---
✅ Hợp nhất hoàn tất. Số lượng sản phẩm trong bảng mới: 282276
Các cột mới được thêm vào hoặc cập nhật: 'joined_category_name', 'category_level'

--- Dữ liệu sau khi hợp nhất (5 dòng đầu) ---
   product_id    product_sku  \
0      416389  4674425006297   
1    32720778  1573715963148   
2     3320515  1988630280650   
3   272532949  7668228112357   
4    22881852  5587108239013   

                                        product_name  \
0  Truyện đọc tiếng Anh - Wordsworth Editions: Th...   
1         Sách tiếng Anh - Principles: Life and Work   
2  Sách thiếu nhi tiếng Anh - Peppa Pig: Little L...   
3                                   Batman: Year One   
4                  English Grammar in Use Book w Ans   

                                     product_url_key  \
0  truyen-doc-tieng-anh-wordsworth-editions-the-l...   
1  sach-tieng-anh-principles-life-and-work-p32720778   
2  sach-thieu-nhi-tieng-anh-peppa-pig-little-libr...   
3

<ipython-input-4-76ac45e06122>:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['joined_category_name'].fillna("Không rõ", inplace=True)
<ipython-input-4-76ac45e06122>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [ ]:
#@title 5. Xây dựng Cột Mô tả cho RAG (`rag_description`)

print("--- Bắt đầu tạo cột mô tả cho RAG ---")

# Đảm bảo df_merged đã được tạo ở Cell 4
if 'df_merged' not in globals():
    print("LỖI: DataFrame 'df_merged' chưa được tạo. Vui lòng chạy lại Cell 4.")
else:
    def create_rag_description(row):
        """
        Tạo một đoạn mô tả tự nhiên, giàu thông tin cho từng sản phẩm.
        """
        # Bắt đầu với thông tin cơ bản
        name = str(row['product_name']) if pd.notna(row['product_name']) else "Sản phẩm không có tên"
        author = str(row['author_brand_name']) if pd.notna(row['author_brand_name']) and row['author_brand_name'] != "Không rõ" else ""

        # Ưu tiên 'joined_category_name' nếu có, nếu không thì dùng 'crawled_category_name'
        category = str(row['joined_category_name']) if pd.notna(row['joined_category_name']) and row['joined_category_name'] != "Không rõ" else str(row['crawled_category_name'])

        price = int(row['price']) if pd.notna(row['price']) else 0

        # Tạo mô tả cơ bản
        description = f"Tên sách là '{name}'"
        if author: # Chỉ thêm tác giả nếu có thông tin
            description += f", của tác giả hoặc thương hiệu là '{author}'"

        if category != "Không rõ":
            description += f", thuộc danh mục '{category}'. "
        else:
            description += ". "

        # Thêm thông tin về giá
        description += f"Giá bán hiện tại của sản phẩm này là {price:,} đồng. "

        # Thêm thông tin khuyến mãi nếu có
        original_price = int(row['original_price']) if pd.notna(row['original_price']) else 0
        discount_rate = int(row['discount_rate']) if pd.notna(row['discount_rate']) else 0

        if discount_rate > 0 and original_price > 0:
            description += f"Giá gốc của sản phẩm là {original_price:,} đồng, hiện đang được giảm giá {discount_rate}%. "

        # Thêm thông tin đánh giá và số lượng bán nếu có
        rating = row['rating_average'] if pd.notna(row['rating_average']) else 0
        reviews = int(row['review_count']) if pd.notna(row['review_count']) else 0
        sold = int(row['quantity_sold']) if pd.notna(row['quantity_sold']) else 0

        if reviews > 0:
            description += f"Quyển sách này được khách hàng đánh giá trung bình {rating:.1f} trên 5 sao, dựa trên {reviews} lượt nhận xét. "
        else:
            description += "Hiện tại chưa có đánh giá nào cho sản phẩm này. "

        if sold > 0:
            description += f"Đã có hơn {sold} lượt mua cho sản phẩm này."
        else:
            description += "Chưa có thông tin về số lượng đã bán."

        return description.strip()

    # Áp dụng hàm để tạo cột mới
    df_merged['rag_description'] = df_merged.apply(create_rag_description, axis=1)

    print("✅ Đã tạo xong cột 'rag_description'.")

    # Hiển thị một vài ví dụ để kiểm tra
    print("\n--- VÍ DỤ VỀ CÁC MÔ TẢ ĐÃ TẠO (5 sản phẩm đầu tiên) ---")
    for i in range(min(5, len(df_merged))): # Đảm bảo không vượt quá số dòng
        print(f"\nSản phẩm {df_merged['product_id'].iloc[i]}:")
        print(df_merged['rag_description'].iloc[i])
        print("-" * 30)

    print("\n--- Kiểm tra một vài mô tả ngẫu nhiên khác ---")
    if len(df_merged) > 100: # Chỉ lấy mẫu ngẫu nhiên nếu có đủ dữ liệu
        for i in df_merged.sample(min(3, len(df_merged))).index:
            print(f"\nSản phẩm {df_merged['product_id'].loc[i]} (ngẫu nhiên):")
            print(df_merged['rag_description'].loc[i])
            print("-" * 30)

--- Bắt đầu tạo cột mô tả cho RAG ---
✅ Đã tạo xong cột 'rag_description'.

--- VÍ DỤ VỀ CÁC MÔ TẢ ĐÃ TẠO (5 sản phẩm đầu tiên) ---

Sản phẩm 416389:
Tên sách là 'Truyện đọc tiếng Anh - Wordsworth Editions: The Little Prince', của tác giả hoặc thương hiệu là 'ANTOINE DE SAINT-EXUPÉRY', thuộc danh mục 'English Books'. Giá bán hiện tại của sản phẩm này là 132,610 đồng. Giá gốc của sản phẩm là 149,000 đồng, hiện đang được giảm giá 11%. Quyển sách này được khách hàng đánh giá trung bình 4.7 trên 5 sao, dựa trên 127 lượt nhận xét. Đã có hơn 567 lượt mua cho sản phẩm này.
------------------------------

Sản phẩm 32720778:
Tên sách là 'Sách tiếng Anh - Principles: Life and Work', của tác giả hoặc thương hiệu là 'RAY DALIO', thuộc danh mục 'English Books'. Giá bán hiện tại của sản phẩm này là 956,237 đồng. Giá gốc của sản phẩm là 1,113,750 đồng, hiện đang được giảm giá 14%. Quyển sách này được khách hàng đánh giá trung bình 4.6 trên 5 sao, dựa trên 9 lượt nhận xét. Đã có hơn 62 lượt mua cho sả

In [ ]:
#@title 6. Lưu trữ Bảng Dữ liệu đã xử lý

print("--- Bắt đầu lưu trữ dữ liệu đã xử lý ---")

# Đảm bảo df_merged đã được tạo và có cột rag_description
if 'df_merged' not in globals() or 'rag_description' not in df_merged.columns:
    print("LỖI: DataFrame 'df_merged' hoặc cột 'rag_description' chưa được tạo. Vui lòng chạy lại Cell 4 và Cell 5.")
else:
    # Chọn các cột cần thiết cho mô hình RAG cuối cùng
    # Đảm bảo bao gồm các cột bạn muốn chatbot có thể tham chiếu hoặc hiển thị
    final_columns = [
        'product_id',
        'product_name',
        'author_brand_name',
        'joined_category_name', # Tên danh mục đã join, dễ đọc hơn
        'category_level',       # Level của danh mục
        'price',
        'original_price',
        'discount_rate',
        'rating_average',
        'review_count',
        'quantity_sold',
        'product_url_path',     # Link trực tiếp đến sản phẩm trên Tiki
        'thumbnail_url',        # Link ảnh bìa
        'rag_description'       # Cột mô tả chính cho RAG
    ]

    # Kiểm tra xem tất cả các cột trong final_columns có tồn tại trong df_merged không
    missing_cols = [col for col in final_columns if col not in df_merged.columns]
    if missing_cols:
        print(f"LỖI: Các cột sau không tìm thấy trong df_merged: {missing_cols}")
        print("Vui lòng kiểm tra lại tên cột ở Cell 4 và Cell 5.")
    else:
        df_final = df_merged[final_columns].copy()

        # Loại bỏ các sản phẩm trùng lặp nếu có, dựa trên product_id
        # Giữ lại bản ghi đầu tiên nếu có trùng lặp
        df_final.drop_duplicates(subset=['product_id'], keep='first', inplace=True)

        # Xác định lại đường dẫn DRIVE_PATH nếu chưa có
        if 'DRIVE_PATH' not in globals():
            DRIVE_PATH = '/content/drive/MyDrive/tiki_data_crawled' # Đường dẫn mặc định, cần kiểm tra
            print(f"CẢNH BÁO: Biến DRIVE_PATH chưa được định nghĩa, sử dụng giá trị mặc định: {DRIVE_PATH}")
            print("Hãy đảm bảo đường dẫn này chính xác.")


        PROCESSED_FILE_PATH = os.path.join(DRIVE_PATH, 'tiki_books_processed_for_rag.csv')

        try:
            df_final.to_csv(PROCESSED_FILE_PATH, index=False, encoding='utf-8-sig')
            print(f"\n✅ HOÀN TẤT! Đã lưu {len(df_final)} sản phẩm đã được xử lý vào file:")
            print(PROCESSED_FILE_PATH)
            print("\nBây giờ bạn có thể sử dụng file này để xây dựng Vector Database (FAISS).")
        except Exception as e:
            print(f"LỖI khi lưu file: {e}")
            print(f"Hãy đảm bảo đường dẫn '{DRIVE_PATH}' tồn tại và bạn có quyền ghi vào đó.")

--- Bắt đầu lưu trữ dữ liệu đã xử lý ---

✅ HOÀN TẤT! Đã lưu 180718 sản phẩm đã được xử lý vào file:
/content/drive/MyDrive/tiki_data_crawled/tiki_books_processed_for_rag.csv

Bây giờ bạn có thể sử dụng file này để xây dựng Vector Database (FAISS).


In [ ]:
#@title 7. Cài đặt Thư viện và Tải Dữ liệu đã xử lý
# Cài đặt thư viện sentence-transformers và faiss-cpu
!pip install sentence-transformers faiss-cpu -q

import pandas as pd
import numpy as np
import os
from sentence_transformers import SentenceTransformer
import faiss # Thư viện FAISS

# Kết nối lại Google Drive nếu phiên làm việc bị ngắt (thường không cần nếu các cell trước đã chạy)
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True) # force_remount=True nếu cần mount lại

# --- !!! QUAN TRỌNG: Đảm bảo đường dẫn này ĐÚNG !!! ---
DRIVE_PATH = '/content/drive/MyDrive/tiki_data_crawled'
PROCESSED_FILE_PATH = os.path.join(DRIVE_PATH, 'tiki_books_processed_for_rag.csv')
# ---------------------------------------------------------

try:
    df_rag_input = pd.read_csv(PROCESSED_FILE_PATH)
    print(f"✅ Đã tải thành công file dữ liệu đã xử lý! Có {len(df_rag_input)} sản phẩm.")
    print("Kiểm tra 5 dòng đầu:")
    print(df_rag_input.head())
    # Kiểm tra cột rag_description
    if 'rag_description' not in df_rag_input.columns:
        raise ValueError("LỖI: Cột 'rag_description' không tồn tại trong file đã tải. Vui lòng kiểm tra lại Cell 5 và 6.")
    # Xử lý trường hợp rag_description có thể bị đọc thành float nếu toàn NaN (ít khả năng sau khi xử lý)
    df_rag_input['rag_description'] = df_rag_input['rag_description'].astype(str)


except FileNotFoundError:
    print(f"❌ LỖI: Không tìm thấy file '{PROCESSED_FILE_PATH}'.")
    print("Vui lòng kiểm tra lại biến DRIVE_PATH và tên file, sau đó chạy lại Cell 6 nếu cần.")
except Exception as e:
    print(f"❌ LỖI khi tải file: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.4 MB/s eta 0:00:00
✅ Đã tải thành công file dữ liệu đã xử lý! Có 180718 sản phẩm.
Kiểm tra 5 dòng đầu:
   product_id                           

In [ ]:
# #@title 8. Tạo Embeddings cho Mô tả Sản phẩm

# # Đảm bảo df_rag_input đã được tải
# if 'df_rag_input' not in globals() or df_rag_input.empty:
#     print("LỖI: DataFrame 'df_rag_input' chưa được tải hoặc rỗng. Vui lòng chạy lại Cell 7.")
# else:
#     print("--- Bắt đầu tạo embeddings ---")

#     # --- !!! THAY ĐỔI TÊN MODEL TẠI ĐÂY NẾU BẠN CÓ TÊN "HALONG EMBEDDING" CỤ THỂ !!! ---
#     # Ví dụ: MODEL_NAME = 'NhomHalong/halong-embedding-v1' (đây là tên giả định)
#     MODEL_NAME = 'VoVanPhuc/sup-SimCSE-VietNamese-phobert-base'
#     # MODEL_NAME = 'bkai-foundation-models/vietnamese-bi-encoder' # Một lựa chọn mạnh khác
#     # ------------------------------------------------------------------------------------

#     print(f"Đang tải mô hình embedding: {MODEL_NAME}...")
#     # Tải mô hình, có thể mất vài phút tùy thuộc vào kích thước mô hình và tốc độ mạng
#     try:
#         embedding_model = SentenceTransformer(MODEL_NAME)
#         print(f"✅ Đã tải thành công mô hình: {MODEL_NAME}")
#     except Exception as e:
#         print(f"❌ LỖI khi tải mô hình embedding '{MODEL_NAME}': {e}")
#         print("Vui lòng kiểm tra lại tên mô hình. Nếu bạn muốn dùng mô hình khác, hãy cập nhật biến MODEL_NAME.")
#         embedding_model = None

#     if embedding_model:
#         # Lấy danh sách các mô tả sản phẩm
#         # Đảm bảo không có giá trị NaN trong cột này (đã xử lý ở cell trước bằng astype(str))
#         # và chuyển tất cả về string để đảm bảo
#         df_rag_input['rag_description'] = df_rag_input['rag_description'].astype(str)
#         product_descriptions = df_rag_input['rag_description'].tolist()

#         print(f"\nBắt đầu tạo embeddings cho {len(product_descriptions)} mô tả sản phẩm...")
#         print("Quá trình này có thể mất nhiều thời gian tùy thuộc vào số lượng sản phẩm và sức mạnh của Colab runtime...")

#         # Tạo embeddings
#         # batch_size có thể điều chỉnh để tối ưu tốc độ và bộ nhớ
#         # show_progress_bar=True để xem tiến trình
#         try:
#             # Đối với một số mô hình, việc giới hạn độ dài max_seq_length có thể cần thiết
#             # Ví dụ: max_seq_length = 256 hoặc 512
#             # embedding_model.max_seq_length = 256
#             product_embeddings = embedding_model.encode(product_descriptions, show_progress_bar=True, batch_size=32) # Giảm batch_size nếu gặp vấn đề OOM
#             print(f"\n✅ Đã tạo thành công {len(product_embeddings)} embeddings.")
#             print(f"Kích thước của mỗi embedding: {product_embeddings.shape[1]}") # (số lượng sản phẩm, số chiều của vector)
#         except Exception as e:
#             print(f"❌ LỖI trong quá trình tạo embeddings: {e}")
#             product_embeddings = None
#     else:
#         product_embeddings = None

In [ ]:
# #@title 9. Xây dựng và Lưu FAISS Index

# # Đảm bảo product_embeddings đã được tạo ở Cell 8
# if 'product_embeddings' not in globals() or product_embeddings is None:
#     print("LỖI: Biến 'product_embeddings' chưa được tạo hoặc có lỗi. Vui lòng chạy lại Cell 8.")
# elif 'df_rag_input' not in globals() or df_rag_input.empty:
#     print("LỖI: DataFrame 'df_rag_input' chưa được tải hoặc rỗng. Vui lòng chạy lại Cell 7 (tải dữ liệu đã xử lý).")
# else:
#     print("--- Bắt đầu xây dựng FAISS Index ---")

#     # Chuyển đổi embeddings sang kiểu float32 (FAISS yêu cầu)
#     product_embeddings_faiss = np.array(product_embeddings).astype('float32')

#     # Lấy số chiều của vector embedding
#     embedding_dimension = product_embeddings_faiss.shape[1]

#     # Tạo FAISS index
#     # IndexFlatL2 là một index đơn giản, sử dụng L2 distance (Euclidean distance)
#     # Phù hợp cho hầu hết các trường hợp bắt đầu.
#     print(f"Khởi tạo FAISS index với chiều vector là: {embedding_dimension}")
#     faiss_index = faiss.IndexFlatL2(embedding_dimension)

#     # Thêm các vector embeddings vào index
#     print(f"Đang thêm {len(product_embeddings_faiss)} vector vào FAISS index...")
#     faiss_index.add(product_embeddings_faiss)

#     print(f"✅ Đã xây dựng xong FAISS index. Tổng số vector trong index: {faiss_index.ntotal}")

#     # Lưu FAISS index vào file
#     # Đảm bảo biến DRIVE_PATH đã được định nghĩa ở Cell 7
#     if 'DRIVE_PATH' not in globals():
#         DRIVE_PATH = '/content/drive/MyDrive/tiki_data_crawled' # Cần đảm bảo đường dẫn này đúng
#         print(f"CẢNH BÁO: Biến DRIVE_PATH chưa được định nghĩa, sử dụng giá trị mặc định: {DRIVE_PATH}")

#     FAISS_INDEX_FILE_PATH = os.path.join(DRIVE_PATH, 'tiki_books_faiss.index')
#     # Cũng nên lưu lại DataFrame df_rag_input (hoặc chỉ các cột cần thiết như product_id và rag_description)
#     # tương ứng với index này, vì FAISS index chỉ lưu vector, không lưu nội dung gốc.
#     # File tiki_books_processed_for_rag.csv đã làm việc này.

#     try:
#         faiss.write_index(faiss_index, FAISS_INDEX_FILE_PATH)
#         print(f"\n✅ Đã lưu FAISS index vào file:")
#         print(FAISS_INDEX_FILE_PATH)
#         print("\nLƯU Ý QUAN TRỌNG:")
#         print("1. File FAISS index này chứa các vector đã được tính toán.")
#         print("2. File CSV đã xử lý ('tiki_books_processed_for_rag.csv') chứa thông tin sản phẩm gốc tương ứng.")
#         print("   Bạn cần cả hai file này cho các bước RAG tiếp theo.")
#         print("3. Trong các session Colab sau, bạn có thể tải trực tiếp file index này mà không cần chạy lại Cell 8 (tạo embeddings).")

#     except Exception as e:
#         print(f"❌ LỖI khi lưu FAISS index: {e}")
#         print(f"Hãy đảm bảo đường dẫn '{DRIVE_PATH}' tồn tại và bạn có quyền ghi vào đó.")

In [ ]:
#@title 9.1. Tải FAISS Index và Dữ liệu Sản phẩm (CHO CÁC LẦN CHẠY SAU)

import pandas as pd
import numpy as np
import os
import faiss # Thư viện FAISS
# from sentence_transformers import SentenceTransformer # Không cần tải lại model nếu chỉ load index

# Kết nối lại Google Drive nếu phiên làm việc bị ngắt
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# --- !!! QUAN TRỌNG: Đảm bảo các đường dẫn này ĐÚNG !!! ---
DRIVE_PATH = '/content/drive/MyDrive/tiki_data_crawled' # Đường dẫn tới thư mục chứa dữ liệu
PROCESSED_FILE_PATH = os.path.join(DRIVE_PATH, 'tiki_books_processed_for_rag.csv')
FAISS_INDEX_FILE_PATH = os.path.join(DRIVE_PATH, 'tiki_books_faiss.index')
# ---------------------------------------------------------

print("--- Bắt đầu tải lại dữ liệu và FAISS index đã lưu ---")

# 1. Tải lại DataFrame chứa thông tin sản phẩm (đặc biệt là cột 'rag_description')
try:
    df_rag_input = pd.read_csv(PROCESSED_FILE_PATH)
    # Đảm bảo cột 'rag_description' được đọc là string
    df_rag_input['rag_description'] = df_rag_input['rag_description'].astype(str)
    print(f"✅ Đã tải thành công file dữ liệu sản phẩm đã xử lý! Có {len(df_rag_input)} sản phẩm.")
    # Bạn có thể muốn giữ lại product_id để map kết quả tìm kiếm
    # product_ids = df_rag_input['product_id'].tolist()
    # product_descriptions_for_retrieval = df_rag_input['rag_description'].tolist()
except FileNotFoundError:
    print(f"❌ LỖI: Không tìm thấy file sản phẩm đã xử lý '{PROCESSED_FILE_PATH}'.")
    print("Vui lòng đảm bảo file này tồn tại hoặc chạy lại các bước tiền xử lý (Cell 2-6).")
    df_rag_input = None
except Exception as e:
    print(f"❌ LỖI khi tải file sản phẩm: {e}")
    df_rag_input = None

# 2. Tải lại FAISS index
if df_rag_input is not None: # Chỉ tải index nếu đã tải được dữ liệu sản phẩm
    try:
        faiss_index = faiss.read_index(FAISS_INDEX_FILE_PATH)
        print(f"\n✅ Đã tải thành công FAISS index từ file: {FAISS_INDEX_FILE_PATH}")
        print(f"Tổng số vector trong index đã tải: {faiss_index.ntotal}")
        if faiss_index.ntotal != len(df_rag_input):
            print(f"⚠️ CẢNH BÁO: Số lượng vector trong FAISS index ({faiss_index.ntotal}) không khớp với số lượng sản phẩm trong file CSV ({len(df_rag_input)}).")
            print("Điều này có thể xảy ra nếu file CSV hoặc file index không được cập nhật đồng bộ.")
    except FileNotFoundError:
        print(f"❌ LỖI: Không tìm thấy file FAISS index '{FAISS_INDEX_FILE_PATH}'.")
        print("Vui lòng đảm bảo file này tồn tại hoặc chạy lại Cell 8 và Cell 9 để tạo và lưu index.")
        faiss_index = None
    except Exception as e:
        print(f"❌ LỖI khi tải FAISS index: {e}")
        faiss_index = None
else:
    faiss_index = None

if df_rag_input is not None and faiss_index is not None:
    print("\n[INFO] Dữ liệu sản phẩm và FAISS index đã sẵn sàng để sử dụng cho RAG.")
else:
    print("\n[ERROR] Không thể tải đủ dữ liệu cần thiết. Vui lòng kiểm tra các lỗi ở trên.")

--- Bắt đầu tải lại dữ liệu và FAISS index đã lưu ---
✅ Đã tải thành công file dữ liệu sản phẩm đã xử lý! Có 180718 sản phẩm.

✅ Đã tải thành công FAISS index từ file: /content/drive/MyDrive/tiki_data_crawled/tiki_books_faiss.index
Tổng số vector trong index đã tải: 180718

[INFO] Dữ liệu sản phẩm và FAISS index đã sẵn sàng để sử dụng cho RAG.


In [ ]:
#@title 10. Thiết lập Gemini API và Xây dựng Pipeline RAG (Cập nhật: Mở rộng truy vấn & Lựa chọn thông minh)

import google.generativeai as genai
from google.colab import userdata # Để lấy API key từ Colab Secrets
import time
import numpy as np
from sentence_transformers import SentenceTransformer
import re # Thêm thư viện regex để xử lý text

# --- 1. Cấu hình API Key cho Gemini ---
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        raise ValueError("GEMINI_API_KEY không được tìm thấy trong Colab Secrets.")
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Đã cấu hình thành công Gemini API key từ Colab Secrets.")
except Exception as e:
    print(f"⚠️ LỖI: Không thể lấy GEMINI_API_KEY từ Colab Secrets. {e}")
    print("Vui lòng thêm API key vào Colab Secrets (biểu tượng chìa khóa bên trái) với tên 'GEMINI_API_KEY'.")
    GEMINI_API_KEY = None

# --- 2. Khởi tạo Mô hình Gemini ---
GEMINI_MODEL_NAME = 'gemini-1.5-flash-latest'
gemini_llm_model = None

if GEMINI_API_KEY:
    try:
        gemini_llm_model = genai.GenerativeModel(GEMINI_MODEL_NAME)
        print(f"✅ Đã khởi tạo thành công mô hình Gemini: {GEMINI_MODEL_NAME}")
    except Exception as e:
        print(f"❌ LỖI khi khởi tạo mô hình Gemini: {e}")
else:
    print("❌ Không có API Key, không thể khởi tạo mô hình Gemini.")

# --- 3. Kiểm tra và Tải lại các thành phần RAG nếu cần ---
can_run_rag = True

if 'embedding_model' not in globals() or embedding_model is None:
    print("Thông báo: Biến 'embedding_model' chưa được tạo hoặc chưa được tải lại. Đang thử tải lại...")
    MODEL_NAME_FOR_RAG = 'VoVanPhuc/sup-SimCSE-VietNamese-phobert-base'
    try:
        embedding_model = SentenceTransformer(MODEL_NAME_FOR_RAG)
        print(f"✅ (Tải lại tự động) Đã tải thành công mô hình embedding: {MODEL_NAME_FOR_RAG}")
    except Exception as e:
        print(f"❌ LỖI khi tự động tải lại mô hình embedding '{MODEL_NAME_FOR_RAG}': {e}")
        embedding_model = None
        can_run_rag = False

if 'faiss_index' not in globals() or faiss_index is None:
    print("LỖI: Biến 'faiss_index' chưa được tải hoặc tạo. Vui lòng chạy Cell 9 (nếu vừa tạo embeddings) hoặc Cell 9.1 (nếu tải index đã lưu).")
    can_run_rag = False

if 'df_rag_input' not in globals() or df_rag_input.empty:
    print("LỖI: DataFrame 'df_rag_input' chưa được tải. Vui lòng chạy Cell 7 (tải dữ liệu đã xử lý) hoặc Cell 9.1.")
    can_run_rag = False

def get_query_embedding(query_text):
    if not can_run_rag or embedding_model is None:
        print("Lỗi: embedding_model chưa sẵn sàng để tạo embedding cho truy vấn.")
        return None
    try:
        return embedding_model.encode([query_text])[0].astype('float32')
    except Exception as e:
        print(f"Lỗi khi tạo embedding cho truy vấn: {e}")
        return None

def search_faiss_index(query_embedding, top_k=5):
    if not can_run_rag or faiss_index is None:
        print("Lỗi: faiss_index chưa sẵn sàng.")
        return np.array([]), np.array([])
    if query_embedding is None:
        return np.array([]), np.array([])
    try:
        query_vector = np.array([query_embedding])
        distances, indices = faiss_index.search(query_vector, top_k)
        return distances[0], indices[0]
    except Exception as e:
        print(f"Lỗi khi tìm kiếm trong FAISS index: {e}")
        return np.array([]), np.array([])

def generate_gemini_response(prompt_text):
    if not can_run_rag or gemini_llm_model is None:
        return "Xin lỗi, mô hình AI hiện tại chưa sẵn sàng để trả lời."
    try:
        safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]
        generation_config = genai.types.GenerationConfig(
            candidate_count=1,
            temperature=0.7 # Có thể thử nghiệm với temperature
        )
        response = gemini_llm_model.generate_content(
            prompt_text,
            generation_config=generation_config,
            safety_settings=safety_settings
        )
        return response.text
    except Exception as e:
        print(f"Lỗi khi gọi Gemini API: {e}")
        return f"Xin lỗi, đã có lỗi xảy ra khi kết nối với dịch vụ AI. Chi tiết: {str(e)}"

# --- NEW: Function to generate expanded queries ---
def generate_expanded_queries(user_query, num_queries=3):
    if gemini_llm_model is None:
        print("Lỗi: Mô hình Gemini chưa sẵn sàng để mở rộng truy vấn.")
        return [user_query] # Fallback to original query

    prompt = f"""Người dùng muốn tìm kiếm sách với câu hỏi sau: "{user_query}".
    Hãy phân tích câu hỏi này và tạo ra TỐI ĐA {num_queries} cụm từ/câu hỏi tìm kiếm có thể dùng để truy vấn cơ sở dữ liệu sách.
    Mỗi cụm từ/câu hỏi nên tập trung vào các khía cạnh khác nhau của ý định tìm kiếm của người dùng, hoặc các từ khóa liên quan.
    Chỉ trả lời bằng một danh sách các cụm từ/câu hỏi, mỗi cụm từ/câu hỏi trên một dòng mới.
    Ví dụ:
    Nếu người dùng hỏi: "Sách về kinh tế"
    Trả lời:
    sách kinh tế
    sách tài chính
    sách đầu tư
    ---
    Nếu người dùng hỏi: "Truyện thiếu nhi"
    Trả lời:
    truyện thiếu nhi
    sách cho trẻ em
    sách tuổi thơ
    ---
    Nếu người dùng hỏi: "{user_query}"
    Trả lời:
    """
    try:
        response_text = generate_gemini_response(prompt)
        # Split by newline and clean up each query
        expanded_queries = [q.strip() for q in response_text.split('\n') if q.strip()]
        # Ensure original query is always included
        if user_query not in expanded_queries:
            expanded_queries.insert(0, user_query)
        return expanded_queries[:num_queries] # Limit to num_queries
    except Exception as e:
        print(f"Lỗi khi tạo truy vấn mở rộng: {e}")
        return [user_query] # Fallback to original query

def rag_pipeline(user_query, top_k_retrieval=15): # Tăng top_k_retrieval để LLM có nhiều ngữ cảnh hơn để chọn lọc
    if not can_run_rag:
        if 'embedding_model' not in globals() or embedding_model is None:
             print("LỖI CUỐI CÙNG: embedding_model vẫn chưa sẵn sàng. Không thể chạy RAG pipeline.")
             return "Hệ thống RAG chưa sẵn sàng do không tải được embedding model. Vui lòng kiểm tra lại.", []
        elif faiss_index is None or df_rag_input.empty:
             return "Hệ thống RAG chưa sẵn sàng do thiếu FAISS index hoặc dữ liệu sản phẩm. Vui lòng kiểm tra các bước trước.", []

    print(f"\nĐang xử lý truy vấn: '{user_query}'")

    # --- NEW: Query Expansion ---
    print("  Đang tạo truy vấn mở rộng...")
    expanded_queries = generate_expanded_queries(user_query, num_queries=3)
    print(f"  Các truy vấn mở rộng: {expanded_queries}")

    all_retrieved_product_ids = set()
    all_retrieved_products_with_distance = []

    for q_text in expanded_queries:
        query_embed_start_time = time.time()
        query_embedding = get_query_embedding(q_text)
        if query_embedding is None:
            print(f"  Không thể tạo embedding cho truy vấn mở rộng: '{q_text}'. Bỏ qua.")
            continue
        embed_time = time.time() - query_embed_start_time
        print(f"  Thời gian tạo embedding cho '{q_text}': {embed_time:.4f} giây")

        search_start_time = time.time()
        # Tìm kiếm với top_k_retrieval cho mỗi truy vấn mở rộng
        distances, indices = search_faiss_index(query_embedding, top_k=top_k_retrieval)
        search_time = time.time() - search_start_time
        print(f"  Thời gian tìm kiếm FAISS cho '{q_text}': {search_time:.4f} giây. Tìm thấy {len(indices)} kết quả.")

        for i, doc_index in enumerate(indices):
            try:
                doc_id = df_rag_input.iloc[int(doc_index)]['product_id']
                if doc_id not in all_retrieved_product_ids:
                    all_retrieved_product_ids.add(doc_id)
                    all_retrieved_products_with_distance.append({
                        'product_id': doc_id,
                        'distance': distances[i],
                        'original_index': int(doc_index) # Lưu lại index gốc để truy xuất df_rag_input
                    })
            except Exception as e:
                print(f"Lỗi khi xử lý sản phẩm truy xuất từ index {doc_index}: {e}")
                continue

    # Sắp xếp lại các sản phẩm đã truy xuất theo khoảng cách (distance) tăng dần
    all_retrieved_products_with_distance.sort(key=lambda x: x['distance'])

    # Lấy thông tin chi tiết cho các sản phẩm đã truy xuất (tối đa top_k_retrieval sau khi hợp nhất)
    final_retrieved_products_info = []
    for item in all_retrieved_products_with_distance[:top_k_retrieval]:
        final_retrieved_products_info.append(df_rag_input.iloc[item['original_index']])

    if not final_retrieved_products_info:
        no_context_prompt = f"""Người dùng hỏi: "{user_query}".
        Là một trợ lý AI cho trang bán sách Tiki, hãy thông báo rằng bạn không tìm thấy thông tin sản phẩm cụ thể nào
        trong cơ sở dữ liệu phù hợp với yêu cầu này.
        Đề nghị người dùng thử một truy vấn khác hoặc cung cấp thêm chi tiết.
        Trả lời bằng tiếng Việt.
        Trợ lý:"""
        print("  Không tìm thấy sản phẩm nào trong cơ sở dữ liệu. Tạo câu trả lời chung...")
        response_text = generate_gemini_response(no_context_prompt)
        return response_text, []

    # --- Cập nhật format_retrieved_context để bao gồm nhiều thông tin hơn cho LLM ---
    context = "Dưới đây là thông tin về một số sản phẩm sách có thể liên quan từ cơ sở dữ liệu Tiki:\n\n"
    for i, product_info in enumerate(final_retrieved_products_info):
        name = product_info.get('product_name', 'N/A')
        author = product_info.get('author_brand_name', 'Không rõ')
        price = product_info.get('price', 0)
        quantity_sold = product_info.get('quantity_sold', 0)
        rating_average = product_info.get('rating_average', 0.0)
        review_count = product_info.get('review_count', 0)
        product_link = product_info.get('product_url_path', '#')
        description_for_llm = product_info.get('rag_description', 'Không có mô tả.')
        # distance = product_info.get('distance', np.nan) # Distance đã được dùng để sắp xếp

        context += f"--- Sản phẩm {i+1} (ID: {product_info['product_id']}) ---\n"
        context += f"   Tên sách: {name}\n"
        context += f"   Tác giả/Thương hiệu: {author}\n"
        context += f"   Giá bán: {price:,} đồng\n"
        context += f"   Số lượt mua ước tính: {quantity_sold}\n"
        context += f"   Đánh giá trung bình: {rating_average:.1f} sao ({review_count} lượt)\n"
        context += f"   Link sản phẩm: {product_link}\n"
        context += f"   Mô tả chi tiết: {description_for_llm}\n\n"

    # --- PROMPT ĐƯỢC CẬP NHẬT ĐỂ GIỚI THIỆU 3 SÁCH VÀ LỰA CHỌN THÔNG MINH HƠN ---
    final_prompt = f"""Bạn là một trợ lý AI chuyên nghiệp và thân thiện của trang web bán sách Tiki.
Nhiệm vụ của bạn là trả lời câu hỏi của người dùng một cách chính xác và hữu ích, CHỈ DỰA TRÊN thông tin sản phẩm được cung cấp trong phần [NGỮ CẢNH SẢN PHẨM].
Tuyệt đối không được bịa đặt hoặc sử dụng kiến thức bên ngoài ngữ cảnh này.

HƯỚNG DẪN TRẢ LỜI:
1.  Đọc kỹ câu hỏi của người dùng: "{user_query}"
2.  Xem xét cẩn thận thông tin trong [NGỮ CẢNH SẢN PHẨM].
3.  Mục tiêu là giới thiệu TỐI ĐA 3 quyển sách phù hợp nhất với câu hỏi của người dùng từ ngữ cảnh được cung cấp.
4.  **KHI LỰA CHỌN SÁCH ĐỂ GIỚI THIỆU, hãy ưu tiên các tiêu chí sau (theo thứ tự ưu tiên):**
    * **Độ liên quan cao nhất** đến câu hỏi.
    * **Độ phổ biến cao:** Ưu tiên các sách có 'Số lượt mua ước tính' cao và/hoặc 'Đánh giá trung bình' cao với nhiều lượt nhận xét.
    * **Tránh trùng lặp:** Tuyệt đối không giới thiệu các phiên bản khác nhau của CÙNG MỘT TỰA SÁCH (ví dụ: cùng tên sách nhưng khác nhà xuất bản hoặc bìa). Hãy chọn phiên bản tốt nhất (ví dụ: có nhiều lượt mua, đánh giá cao, hoặc giá tốt hơn) làm đại diện.
    * **Đa dạng thể loại/tác giả (nếu có thể):** Nếu có nhiều lựa chọn tốt, hãy cố gắng giới thiệu sách từ các tác giả hoặc thể loại hơi khác nhau để cung cấp nhiều lựa chọn hơn cho người dùng.
5.  Với MỖI quyển sách bạn chọn để giới thiệu, hãy trình bày các thông tin sau một cách rõ ràng và mạch lạc:
    * Tên sách đầy đủ.
    * Tác giả hoặc thương hiệu.
    * Một đoạn mô tả ngắn gọn và hấp dẫn về sách (dựa vào thông tin trong "Mô tả chi tiết" của sản phẩm đó).
    * Số lượt mua ước tính (nếu có thông tin và lớn hơn 0).
    * Giá bán hiện tại.
    * Link sản phẩm.
6.  Hãy trình bày thông tin của từng quyển sách một cách riêng biệt, có thể dùng gạch đầu dòng hoặc đánh số.
7.  Nếu sau khi xem xét kỹ ngữ cảnh, bạn chỉ tìm thấy 1 hoặc 2 quyển sách thực sự phù hợp, thì chỉ giới thiệu số lượng đó.
8.  Nếu người dùng hỏi một chi tiết cụ thể mà không có trong mô tả sản phẩm, hãy trả lời rằng thông tin đó không có sẵn.
9.  Nếu không có sản phẩm nào trong ngữ cảnh thực sự phù hợp với câu hỏi, hãy lịch sự thông báo và có thể gợi ý họ thử tìm kiếm với từ khóa khác.
10. Trả lời bằng tiếng Việt, giọng điệu tự nhiên, thân thiện và kết thúc một cách lịch sự.

[NGỮ CẢNH SẢN PHẨM]
{context}
[HẾT NGỮ CẢNH SẢN PHẨM]

Câu trả lời của trợ lý (bằng tiếng Việt, giới thiệu tối đa 3 sách nếu phù hợp, mỗi sách có đủ thông tin yêu cầu):
"""
    # print("\n--- Prompt cuối cùng gửi đến Gemini ---") # Bỏ comment để debug
    # print(final_prompt)

    gen_response_start_time = time.time()
    response_text = generate_gemini_response(final_prompt)
    generation_time = time.time() - gen_response_start_time
    print(f"  Thời gian sinh câu trả lời bởi Gemini: {generation_time:.4f} giây")

    retrieved_products_info_for_output = []
    # Lấy thông tin của tất cả các sản phẩm đã truy xuất (sau khi deduplicate và sắp xếp)
    # để hiển thị trong phần "sản phẩm tham khảo" nếu cần
    if final_retrieved_products_info:
        for product_data_series in final_retrieved_products_info:
            try:
                product_dict = product_data_series[['product_id', 'product_name', 'author_brand_name', 'price', 'quantity_sold', 'product_url_path']].to_dict()
                product_dict['rag_description_snippet'] = product_data_series['rag_description'][:150] + '...' if len(product_data_series['rag_description']) > 150 else product_data_series['rag_description']
                retrieved_products_info_for_output.append(product_dict)
            except Exception as e_info:
                print(f"Lỗi khi lấy thông tin sản phẩm truy xuất cho output: {e_info}")

    return response_text, retrieved_products_info_for_output

if can_run_rag and gemini_llm_model:
    if 'embedding_model' not in globals() or embedding_model is None: # Kiểm tra lại
        print("\n❌ Pipeline RAG CHƯA sẵn sàng do 'embedding_model' không được tải thành công.")
    else:
        print("\n✅ Pipeline RAG đã sẵn sàng để thử nghiệm!")
else:
    print("\n❌ Pipeline RAG CHƯA sẵn sàng. Vui lòng kiểm tra các lỗi được thông báo ở trên và đảm bảo tất cả các thành phần cần thiết đã được khởi tạo/tải đúng cách.")


✅ Đã cấu hình thành công Gemini API key từ Colab Secrets.
✅ Đã khởi tạo thành công mô hình Gemini: gemini-1.5-flash-latest
Thông báo: Biến 'embedding_model' chưa được tạo hoặc chưa được tải lại. Đang thử tải lại...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

✅ (Tải lại tự động) Đã tải thành công mô hình embedding: VoVanPhuc/sup-SimCSE-VietNamese-phobert-base

✅ Pipeline RAG đã sẵn sàng để thử nghiệm!


In [ ]:
#@title 11. Thử nghiệm Chatbot RAG Tiki (Đã sửa lỗi TypeError và tăng top_k_retrieval)

# Đảm bảo các hàm và biến cần thiết đã được định nghĩa và khởi tạo
# Cũng cần đảm bảo Cell 10 đã được chạy trong session này
if 'can_run_rag' not in globals() or not can_run_rag:
    print("LỖI: Hệ thống RAG chưa sẵn sàng. Vui lòng chạy lại các cell trước, đặc biệt là Cell 10.")
elif 'rag_pipeline' not in globals():
    print("LỖI: Hàm 'rag_pipeline' chưa được định nghĩa. Vui lòng chạy lại Cell 10.")
else:
    print("--- CHATBOT TIKI SÁCH ĐÃ SẴN SÀNG ---")
    print("Gõ 'quit' hoặc 'exit' để thoát.")
    print("------------------------------------")

    while True:
        user_query = input("Bạn hỏi: ")
        if user_query.lower() in ['quit', 'exit']:
            print("Cảm ơn bạn đã sử dụng Chatbot Tiki Sách!")
            break
        if not user_query.strip():
            continue

        # SỬA LỖI Ở ĐÂY: Đổi 'top_k' thành 'top_k_retrieval'
        # TĂNG top_k_retrieval để LLM có nhiều sản phẩm hơn để chọn lọc thông minh
        answer, retrieved_docs_info = rag_pipeline(user_query, top_k_retrieval=15) # Tăng từ 5 lên 15 hoặc hơn

        print(f"\nChatbot Tiki: {answer}")

        # Phần hiển thị thông tin sản phẩm tham khảo (nếu bạn muốn giữ lại để debug)
        if retrieved_docs_info:
            print("\n  --- Thông tin các sản phẩm đã được sử dụng để trả lời (tham khảo) ---")
            for i, doc_info in enumerate(retrieved_docs_info):
                print(f"    Sản phẩm {i+1}:")
                print(f"      ID: {doc_info.get('product_id', 'N/A')}") # Sử dụng .get() để tránh lỗi nếu key thiếu
                print(f"      Tên: {doc_info.get('product_name', 'N/A')}")
                # print(f"      Mô tả dùng cho RAG: {doc_info.get('rag_description_snippet', 'N/A')}") # Bỏ comment nếu muốn xem mô tả snippet
            print("  --------------------------------------------------------------------")
        else:
            if isinstance(answer, str) and \
               "không tìm thấy sản phẩm nào phù hợp" not in answer.lower() and \
               "không tìm thấy thông tin sản phẩm cụ thể" not in answer.lower() and \
               "hệ thống rag chưa sẵn sàng" not in answer.lower():
                 print("  (Không có sản phẩm cụ thể nào được truy xuất từ cơ sở dữ liệu cho câu trả lời này, hoặc có lỗi trong pipeline)")
        print("\n------------------------------------")


--- CHATBOT TIKI SÁCH ĐÃ SẴN SÀNG ---
Gõ 'quit' hoặc 'exit' để thoát.
------------------------------------
Bạn hỏi: Truyện trinh thám hay nhất

Đang xử lý truy vấn: 'Truyện trinh thám hay nhất'
  Đang tạo truy vấn mở rộng...
  Các truy vấn mở rộng: ['Truyện trinh thám hay nhất', 'truyện trinh thám hay nhất', 'sách trinh thám được đánh giá cao']
  Thời gian tạo embedding cho 'Truyện trinh thám hay nhất': 0.7961 giây
  Thời gian tìm kiếm FAISS cho 'Truyện trinh thám hay nhất': 0.0840 giây. Tìm thấy 15 kết quả.
  Thời gian tạo embedding cho 'truyện trinh thám hay nhất': 0.0124 giây
  Thời gian tìm kiếm FAISS cho 'truyện trinh thám hay nhất': 0.0502 giây. Tìm thấy 15 kết quả.
  Thời gian tạo embedding cho 'sách trinh thám được đánh giá cao': 0.0807 giây
  Thời gian tìm kiếm FAISS cho 'sách trinh thám được đánh giá cao': 0.0499 giây. Tìm thấy 15 kết quả.
  Thời gian sinh câu trả lời bởi Gemini: 4.6855 giây

Chatbot Tiki: Chào bạn!  Dựa trên thông tin Tiki cung cấp, mình thấy có một số cuốn 

In [ ]:
#@title 12. Đánh giá và Benchmark Hệ thống RAG (Cập nhật Ground Truth cho 'Sách Kinh Tế')

import pandas as pd
import numpy as np
import time
import re

# --- KIỂM TRA CÁC THÀNH PHẦN CẦN THIẾT ---
print("--- Kiểm tra các thành phần cần thiết cho Benchmark ---")
components_ready = True
# (Giữ nguyên phần kiểm tra components_ready như ở các phiên bản trước)
if 'can_run_rag' not in globals() or not can_run_rag:
    print("LỖI: Biến 'can_run_rag' là False hoặc không tồn tại. Pipeline RAG cơ bản chưa sẵn sàng.")
    components_ready = False
if 'df_rag_input' not in globals() or df_rag_input.empty:
    print("LỖI: DataFrame 'df_rag_input' chưa được tải.")
    components_ready = False
if 'gemini_llm_model' not in globals() or gemini_llm_model is None:
    print("LỖI: Mô hình 'gemini_llm_model' chưa được khởi tạo.")
    components_ready = False
if 'rag_pipeline' not in globals():
    print("LỖI: Hàm 'rag_pipeline' chưa được định nghĩa.")
    components_ready = False
if not components_ready:
    raise RuntimeError("Một hoặc nhiều thành phần RAG cơ bản không sẵn sàng để benchmark. Vui lòng kiểm tra các lỗi ở trên.")
else:
    print("✅ Tất cả các thành phần cần thiết cho benchmark đã sẵn sàng.")

# --- Các hàm đánh giá (calculate_context_precision, calculate_context_recall,
# evaluate_faithfulness_with_llm, evaluate_answer_relevance_with_llm giữ nguyên) ---
# (Giữ nguyên toàn bộ các hàm đánh giá và hàm get_pure_llm_response, traditional_keyword_search, run_rag_benchmark
#  như trong phiên bản rag_benchmark_cell_12_v4_comparisons bạn đã có)
def calculate_context_precision(retrieved_doc_ids, ground_truth_relevant_doc_ids):
    if not ground_truth_relevant_doc_ids: return np.nan
    if not retrieved_doc_ids: return 0.0
    retrieved_set = set(str(id_val) for id_val in retrieved_doc_ids)
    ground_truth_set = set(str(id_val) for id_val in ground_truth_relevant_doc_ids)
    true_positives = len(retrieved_set.intersection(ground_truth_set))
    if not retrieved_set: return 0.0
    precision = true_positives / len(retrieved_set)
    return precision

def calculate_context_recall(retrieved_doc_ids, ground_truth_relevant_doc_ids):
    if not ground_truth_relevant_doc_ids: return np.nan
    if not retrieved_doc_ids: return 0.0 if ground_truth_relevant_doc_ids else np.nan
    retrieved_set = set(str(id_val) for id_val in retrieved_doc_ids)
    ground_truth_set = set(str(id_val) for id_val in ground_truth_relevant_doc_ids)
    if not ground_truth_set: return np.nan if retrieved_set else 1.0
    true_positives = len(retrieved_set.intersection(ground_truth_set))
    recall = true_positives / len(ground_truth_set)
    return recall

def evaluate_faithfulness_with_llm(generated_response, retrieved_context_str):
    if not generated_response or not retrieved_context_str or retrieved_context_str.strip() == "Xin lỗi, tôi không tìm thấy sản phẩm nào phù hợp trong cơ sở dữ liệu để tạo ngữ cảnh.":
        return np.nan
    prompt = f"""Bạn là một giám khảo AI cực kỳ cẩn thận và khách quan. Nhiệm vụ của bạn là đánh giá xem "Câu trả lời" có hoàn toàn TRUNG THỰC với "Ngữ cảnh được cung cấp" hay không. ĐỊNH NGHĨA "TRUNG THỰC": 1. TRUNG THỰC (Trả lời "CÓ"): Nếu TẤT CẢ thông tin, sự kiện, chi tiết trong "Câu trả lời" đều được trích xuất trực tiếp, hoặc là một bản tóm tắt/diễn giải hợp lý và không làm sai lệch ý nghĩa của thông tin có trong "Ngữ cảnh được cung cấp". Câu trả lời không thêm bất kỳ thông tin mới nào không có trong ngữ cảnh. 2. KHÔNG TRUNG THỰC (Trả lời "KHÔNG"): Nếu "Câu trả lời" chứa bất kỳ thông tin, chi tiết, khẳng định nào KHÔNG CÓ trong "Ngữ cảnh được cung cấp", hoặc suy diễn vượt quá những gì ngữ cảnh cho phép, hoặc đưa ra thông tin mâu thuẫn với ngữ cảnh. Kể cả những chi tiết nhỏ tưởng chừng hợp lý nhưng không có trong ngữ cảnh cũng bị coi là không trung thực. VÍ DỤ: - Ngữ cảnh: "Sách A giá 100.000 đồng." - Câu trả lời: "Sách A có giá 100.000 đồng." -> TRUNG THỰC (CÓ) - Câu trả lời: "Giá của Sách A là 100.000 đồng." -> TRUNG THỰC (CÓ) - Câu trả lời: "Sách A rất hay và có giá 100.000 đồng." -> KHÔNG TRUNG THỰC (KHÔNG) nếu "rất hay" không có trong ngữ cảnh. - Câu trả lời: "Sách A giá 100.000 đồng và được xuất bản năm 2023." -> KHÔNG TRUNG THỰC (KHÔNG) nếu "năm 2023" không có trong ngữ cảnh. [NGỮ CẢNH ĐƯỢC CUNG CẤP]\n{retrieved_context_str}\n[HẾT NGỮ CẢNH ĐƯỢC CUNG CẤP]\n\n[CÂU TRẢ LỜI CẦN ĐÁNH GIÁ]\n{generated_response}\n[HẾT CÂU TRẢ LỜI CẦN ĐÁNH GIÁ]\n\nCâu hỏi: Dựa trên định nghĩa và ví dụ trên, "Câu trả lời" có hoàn toàn TRUNG THỰC với "Ngữ cảnh được cung cấp" không? Chỉ trả lời bằng một từ duy nhất: "CÓ" hoặc "KHÔNG"."""
    try:
        evaluation_response = gemini_llm_model.generate_content(prompt)
        decision = evaluation_response.text.strip().upper()
        if "CÓ" == decision: return 1.0
        elif "KHÔNG" == decision: return 0.0
        return np.nan
    except Exception: return np.nan

def evaluate_answer_relevance_with_llm(generated_response, user_query):
    if not generated_response or not user_query: return np.nan
    prompt = f"""Bạn là một giám khảo AI. Đánh giá xem "Câu trả lời" có liên quan và trả lời trực tiếp "Câu hỏi của người dùng" không. [CÂU HỎI CỦA NGƯỜI DÙNG]\n{user_query}\n[HẾT CÂU HỎI CỦA NGƯỜI DÙNG]\n\n[CÂU TRẢ LỜI CẦN ĐÁNH GIÁ]\n{generated_response}\n[HẾT CÂU TRẢ LỜI CẦN ĐÁNH GIÁ]\n\nCâu hỏi: "Câu trả lời" có liên quan và trả lời trực tiếp "Câu hỏi của người dùng" không? Chỉ trả lời "CÓ" hoặc "KHÔNG"."""
    try:
        evaluation_response = gemini_llm_model.generate_content(prompt)
        decision = evaluation_response.text.strip().upper()
        if "CÓ" in decision: return 1.0
        elif "KHÔNG" in decision: return 0.0
        return np.nan
    except Exception: return np.nan

def get_pure_llm_response(user_query):
    if gemini_llm_model is None: return "Lỗi: Mô hình Gemini chưa sẵn sàng."
    prompt = f"""Bạn là một trợ lý AI của trang bán sách Tiki. Hãy trả lời câu hỏi sau của người dùng một cách tốt nhất có thể dựa trên kiến thức chung của bạn về sách và các chủ đề liên quan. Câu hỏi của người dùng: "{user_query}" Trả lời bằng tiếng Việt. Trợ lý Tiki:"""
    try:
        response = gemini_llm_model.generate_content(prompt)
        return response.text.strip()
    except Exception as e: return f"Xin lỗi, tôi không thể xử lý yêu cầu này ngay bây giờ. Lỗi: {e}"

def traditional_keyword_search(user_query, dataframe, search_columns=['product_name', 'rag_description'], top_k=5):
    if dataframe.empty: return []
    keywords = [keyword.strip() for keyword in user_query.lower().split() if len(keyword.strip()) > 2]
    if not keywords: return []
    # Tạo bản sao để tránh SettingWithCopyWarning
    df_copy = dataframe.copy()
    df_copy['temp_score'] = 0
    for col in search_columns:
        if col in df_copy.columns:
            search_series = df_copy[col].astype(str).fillna('').str.lower()
            for keyword in keywords:
                df_copy['temp_score'] += search_series.str.contains(keyword, regex=False).astype(int)
    top_results_df = df_copy[df_copy['temp_score'] > 0].sort_values(by='temp_score', ascending=False).head(top_k)
    retrieved_ids = top_results_df['product_id'].tolist()
    # Không cần drop cột temp_score trên df_copy nữa vì nó là bản sao
    return retrieved_ids

def run_rag_benchmark(test_queries_with_ground_truth, top_k_for_rag_pipeline=5, top_k_for_trad_search=5):
    if not test_queries_with_ground_truth:
        print("Không có test case nào để chạy benchmark.")
        return pd.DataFrame()
    benchmark_results_list = []
    total_test_cases = len(test_queries_with_ground_truth)
    print(f"\n--- BẮT ĐẦU CHẠY BENCHMARK VỚI {total_test_cases} TEST CASE ---")
    for i, test_case in enumerate(test_queries_with_ground_truth):
        query = test_case['query']
        gt_doc_ids = test_case.get('ground_truth_doc_ids', [])
        print(f"\n[BENCHMARK_CASE {i+1}/{total_test_cases}] Đang xử lý truy vấn: '{query}'")

        print("  [RAG_PIPELINE] Đang chạy...")
        rag_answer, rag_retrieved_docs_info = rag_pipeline(query, top_k_retrieval=top_k_for_rag_pipeline)
        rag_retrieved_ids = [doc.get('product_id') for doc in rag_retrieved_docs_info if doc.get('product_id') is not None]
        rag_actual_context_for_eval = ""
        if rag_retrieved_docs_info:
            for doc_info in rag_retrieved_docs_info:
                rag_actual_context_for_eval += f"Sản phẩm ID {doc_info.get('product_id', 'N/A')}: {doc_info.get('rag_description', 'Không có mô tả.')}\n\n"
        rag_precision = calculate_context_precision(rag_retrieved_ids, gt_doc_ids)
        rag_recall = calculate_context_recall(rag_retrieved_ids, gt_doc_ids)
        rag_faithfulness = evaluate_faithfulness_with_llm(rag_answer, rag_actual_context_for_eval.strip())
        rag_answer_relevance = evaluate_answer_relevance_with_llm(rag_answer, query)

        print("  [PURE_LLM_BASELINE] Đang chạy...")
        pure_llm_answer = get_pure_llm_response(query)
        pure_llm_answer_relevance = evaluate_answer_relevance_with_llm(pure_llm_answer, query)

        print("  [TRADITIONAL_SEARCH] Đang chạy...")
        # Truyền df_rag_input (DataFrame gốc) vào traditional_keyword_search
        trad_search_retrieved_ids = traditional_keyword_search(query, df_rag_input, top_k=top_k_for_trad_search)
        trad_search_precision = calculate_context_precision(trad_search_retrieved_ids, gt_doc_ids)
        trad_search_recall = calculate_context_recall(trad_search_retrieved_ids, gt_doc_ids)

        benchmark_results_list.append({
            'query': query,
            'rag_answer': rag_answer, 'rag_retrieved_ids': rag_retrieved_ids,
            'rag_context_precision': rag_precision, 'rag_context_recall': rag_recall,
            'rag_faithfulness': rag_faithfulness, 'rag_answer_relevance': rag_answer_relevance,
            'pure_llm_answer': pure_llm_answer, 'pure_llm_answer_relevance': pure_llm_answer_relevance,
            'trad_search_retrieved_ids': trad_search_retrieved_ids,
            'trad_search_precision': trad_search_precision, 'trad_search_recall': trad_search_recall,
            'ground_truth_doc_ids': gt_doc_ids,
        })
        print(f"  [BENCHMARK_CASE {i+1}] Kết quả:")
        print(f"    RAG - Precision: {rag_precision if pd.notna(rag_precision) else 'N/A'}, Recall: {rag_recall if pd.notna(rag_recall) else 'N/A'}, Faithfulness: {rag_faithfulness if pd.notna(rag_faithfulness) else 'N/A'}, Answer Relevance: {rag_answer_relevance if pd.notna(rag_answer_relevance) else 'N/A'}")
        print(f"    Pure LLM - Answer Relevance: {pure_llm_answer_relevance if pd.notna(pure_llm_answer_relevance) else 'N/A'}")
        print(f"    Trad. Search - Precision: {trad_search_precision if pd.notna(trad_search_precision) else 'N/A'}, Recall: {trad_search_recall if pd.notna(trad_search_recall) else 'N/A'}")
    print("\n--- KẾT THÚC BENCHMARK ---")
    results_df = pd.DataFrame(benchmark_results_list)
    if not results_df.empty:
        print("\n--- KẾT QUẢ BENCHMARK TỔNG HỢP (DataFrame) ---")
        pd.set_option('display.max_colwidth', 70)
        display_cols = ['query', 'rag_answer', 'rag_context_precision', 'rag_context_recall', 'rag_faithfulness', 'rag_answer_relevance', 'pure_llm_answer', 'pure_llm_answer_relevance', 'trad_search_precision', 'trad_search_recall']
        existing_display_cols = [col for col in display_cols if col in results_df.columns]
        print(results_df[existing_display_cols].head(20))
        print("\n--- CÁC CHỈ SỐ TRUNG BÌNH ---")
        # (Giữ nguyên phần tính toán và in các chỉ số trung bình)
        print(f"  RAG System:")
        print(f"    Average Context Precision: {results_df['rag_context_precision'].mean(skipna=True):.4f}" if 'rag_context_precision' in results_df else "    Average Context Precision: N/A")
        print(f"    Average Context Recall: {results_df['rag_context_recall'].mean(skipna=True):.4f}" if 'rag_context_recall' in results_df else "    Average Context Recall: N/A")
        print(f"    Average Faithfulness: {results_df['rag_faithfulness'].mean(skipna=True):.4f}" if 'rag_faithfulness' in results_df else "    Average Faithfulness: N/A")
        print(f"    Average Answer Relevance: {results_df['rag_answer_relevance'].mean(skipna=True):.4f}" if 'rag_answer_relevance' in results_df else "    Average Answer Relevance: N/A")
        print(f"\n  Pure LLM Baseline:")
        print(f"    Average Answer Relevance: {results_df['pure_llm_answer_relevance'].mean(skipna=True):.4f}" if 'pure_llm_answer_relevance' in results_df else "    Average Answer Relevance: N/A")
        print(f"\n  Traditional Search:")
        print(f"    Average Context Precision: {results_df['trad_search_precision'].mean(skipna=True):.4f}" if 'trad_search_precision' in results_df else "    Average Context Precision: N/A")
        print(f"    Average Context Recall: {results_df['trad_search_recall'].mean(skipna=True):.4f}" if 'trad_search_recall' in results_df else "    Average Context Recall: N/A")

    else:
        print("Không có kết quả benchmark nào được tạo ra.")
    return results_df


# --- Chuẩn bị Dữ liệu Test cho Benchmark (CẬP NHẬT VỚI OUTPUT MỚI CỦA BẠN) ---
prioritized_benchmark_test_cases = [
    {
        'query': "Tôi muốn mua sách về kinh tế",
        # Dựa trên output tìm kiếm thủ công của bạn, hãy chọn những ID tốt nhất.
        # Ví dụ gợi ý (BẠN CẦN XÁC MINH VÀ CHỌN LỌC KỸ):
        'ground_truth_doc_ids': [44976191, 270847918, 272268612, 276800052, 1387547, 92010637, 273596356, 77221727, 275056308]
    },
    {
        'query': "Tôi muốn mua sách về tình yêu dưới 200k",
        # Dựa trên output Cell 11, ID 260811420 có vẻ liên quan nhất.
        # Bạn cần tìm thêm các ID khác nếu có, và đảm bảo chúng dưới 200k.
        'ground_truth_doc_ids': [260811420] # VÍ DỤ - CẦN BẠN XÁC MINH VÀ BỔ SUNG
    },
    {
        'query': "Tôi muốn mua sách về tình bạn để tặng cho bạn thân",
        # Dựa trên output Cell 11, ID 96308828 có vẻ liên quan.
        # Bạn cần tìm thêm các ID khác nếu có.
        'ground_truth_doc_ids': [96308828] # VÍ DỤ - CẦN BẠN XÁC MINH VÀ BỔ SUNG
    },
    {
        'query': "Sách về Phật Pháp căn bản", # Giữ lại từ lần trước nếu ground truth ổn
        'ground_truth_doc_ids': [4031951, 4014013, 214655863, 276426008, 172111104]
    },
    {
        'query': "Tôi muốn mua sách về đề tài lịch sử Việt Nam", # Giữ lại từ lần trước nếu ground truth ổn
        'ground_truth_doc_ids': [271959379, 276060307, 177746982, 276527090, 273232899, 249575520]
    }
]

# --- Chạy Benchmark ---
print("\nBắt đầu chạy thử nghiệm benchmark với bộ dữ liệu được cập nhật...")

if prioritized_benchmark_test_cases:
    # Chạy toàn bộ các test case đã được ưu tiên và cập nhật ground truth
    # Để test nhanh, bạn có thể giới hạn số lượng test case, ví dụ: prioritized_benchmark_test_cases[:1]
    benchmark_df_results_final = run_rag_benchmark(prioritized_benchmark_test_cases, top_k_for_rag_pipeline=5, top_k_for_trad_search=5)
else:
    print("Danh sách 'prioritized_benchmark_test_cases' đang rỗng. Không có gì để benchmark.")
    benchmark_df_results_final = pd.DataFrame()

print("\nLƯU Ý QUAN TRỌNG: Kết quả benchmark phụ thuộc rất nhiều vào chất lượng của 'ground_truth_doc_ids'.")
print("Hãy đảm bảo bạn đã xem xét kỹ lưỡng và chọn các ID chính xác cho từng truy vấn.")


In [ ]:
import pandas as pd
df_data_to_search = pd.read_csv('/content/drive/MyDrive/tiki_data_crawled/tiki_books_processed_for_rag.csv')

# Tìm theo tên sản phẩm
kinh_te_books_by_name = df_data_to_search[df_data_to_search['product_name'].str.contains("kinh tế", case=False, na=False)]
print("Sách kinh tế tìm theo tên:")
print(kinh_te_books_by_name[['product_id', 'product_name', 'price', 'joined_category_name']].head())

# Tìm theo tên danh mục (nếu bạn có cột tên danh mục đã join)
kinh_te_books_by_cat = df_data_to_search[df_data_to_search['joined_category_name'].str.contains("kinh tế", case=False, na=False)]
print("\nSách kinh tế tìm theo danh mục:")
print(kinh_te_books_by_cat[['product_id', 'product_name', 'price', 'joined_category_name']].head())

# Hoặc tìm trong rag_description
kinh_te_books_by_desc = df_data_to_search[df_data_to_search['rag_description'].str.contains("kinh tế", case=False, na=False)]
print("\nSách kinh tế tìm theo mô tả RAG:")
print(kinh_te_books_by_desc[['product_id', 'product_name', 'price', 'joined_category_name']].head())

In [ ]:
#@title 14. Tổng quan Thông tin Dataset

import pandas as pd

# --- Hiển thị thông tin về DataFrame Danh mục ---
if 'df_categories' in globals() and not df_categories.empty:
    print("--- THÔNG TIN CHI TIẾT VỀ DATAFRAME DANH MỤC (df_categories) ---")
    print(f"Số lượng danh mục: {len(df_categories)}")
    print("\nThông tin cột:")
    df_categories.info()
    print("\n5 dòng dữ liệu đầu tiên:")
    print(df_categories.head())
    print("\nThống kê mô tả cho các cột số (nếu có):")
    print(df_categories.describe(include=np.number))
    print("\nKiểm tra giá trị thiếu:")
    print(df_categories.isnull().sum())
    print("\nSố lượng giá trị duy nhất cho mỗi cột:")
    for col in df_categories.columns:
        print(f"- {col}: {df_categories[col].nunique()}")
    print("="*50)
else:
    print("WARNING: DataFrame 'df_categories' chưa được tải hoặc rỗng. Hãy chạy lại Cell 2.")

# --- Hiển thị thông tin về DataFrame Sản phẩm đã xử lý (df_rag_input hoặc df_final) ---
# Ưu tiên df_rag_input vì nó được sử dụng trong các bước RAG và benchmark
df_to_show = None
df_name = ""

if 'df_rag_input' in globals() and not df_rag_input.empty:
    df_to_show = df_rag_input
    df_name = "df_rag_input (dữ liệu sản phẩm đã xử lý cho RAG)"
elif 'df_final' in globals() and not df_final.empty: # Fallback cho df_final từ Cell 6
    df_to_show = df_final
    df_name = "df_final (dữ liệu sản phẩm đã xử lý)"
elif 'df_merged' in globals() and not df_merged.empty: # Fallback cho df_merged từ Cell 4/5
    df_to_show = df_merged
    df_name = "df_merged (dữ liệu sản phẩm đã hợp nhất và có rag_description)"
elif 'df_products_cleaned' in globals() and not df_products_cleaned.empty: # Fallback cho df_products_cleaned từ Cell 3
    df_to_show = df_products_cleaned
    df_name = "df_products_cleaned (dữ liệu sản phẩm đã làm sạch cơ bản)"
elif 'df_products' in globals() and not df_products.empty: # Fallback cho df_products thô từ Cell 2
    df_to_show = df_products
    df_name = "df_products (dữ liệu sản phẩm thô)"


if df_to_show is not None:
    print(f"\n--- THÔNG TIN CHI TIẾT VỀ DATAFRAME SẢN PHẨM ({df_name}) ---")
    print(f"Số lượng sản phẩm: {len(df_to_show)}")
    print("\nThông tin cột:")
    df_to_show.info()
    print("\n5 dòng dữ liệu đầu tiên:")
    # Hiển thị các cột quan trọng hơn nếu có quá nhiều cột
    if len(df_to_show.columns) > 15:
        cols_to_display_head = ['product_id', 'product_name', 'author_brand_name', 'joined_category_name', 'price', 'rating_average', 'quantity_sold', 'rag_description']
        # Lọc ra những cột thực sự tồn tại trong df_to_show
        existing_cols_display_head = [col for col in cols_to_display_head if col in df_to_show.columns]
        if not existing_cols_display_head: # Nếu không có cột nào trong list trên, hiển thị tất cả
             existing_cols_display_head = df_to_show.columns.tolist()
        print(df_to_show[existing_cols_display_head].head())
    else:
        print(df_to_show.head())

    print("\nThống kê mô tả cho các cột số:")
    print(df_to_show.describe(include=np.number))
    print("\nKiểm tra giá trị thiếu (cho các cột có thể có giá trị thiếu nhiều):")
    cols_to_check_null = ['author_brand_name', 'quantity_sold', 'category_l3_name', 'category_l4_name', 'short_description']
    existing_cols_check_null = [col for col in cols_to_check_null if col in df_to_show.columns]
    if existing_cols_check_null:
        print(df_to_show[existing_cols_check_null].isnull().sum())
    else:
        print(df_to_show.isnull().sum())

    print("\nSố lượng giá trị duy nhất cho một số cột quan trọng:")
    cols_for_nunique = ['author_brand_name', 'joined_category_name', 'category_level', 'crawled_category_name']
    existing_cols_for_nunique = [col for col in cols_for_nunique if col in df_to_show.columns]
    if existing_cols_for_nunique:
        for col in existing_cols_for_nunique:
            print(f"- {col}: {df_to_show[col].nunique()}")
    print("="*50)
else:
    print("WARNING: Không tìm thấy DataFrame sản phẩm nào đã được xử lý (df_rag_input, df_final, df_merged, df_products_cleaned, df_products).")



In [ ]:
# Đảm bảo df_rag_input (hoặc df_final/df_merged) đã được tải và có cột 'rag_description'
df_source_for_rag_desc = None
if 'df_rag_input' in globals() and 'rag_description' in df_rag_input.columns:
    df_source_for_rag_desc = df_rag_input
elif 'df_final' in globals() and 'rag_description' in df_final.columns:
    df_source_for_rag_desc = df_final
elif 'df_merged' in globals() and 'rag_description' in df_merged.columns:
    df_source_for_rag_desc = df_merged

if df_source_for_rag_desc is not None:
    print("--- Ví dụ về rag_description được tạo ra ---")
    # Lấy 5 mẫu ngẫu nhiên nếu DataFrame đủ lớn, nếu không thì lấy ít hơn
    num_samples = min(5, len(df_source_for_rag_desc))
    if num_samples > 0:
        sample_products = df_source_for_rag_desc.sample(num_samples)
        for index, row in sample_products.iterrows():
            print(f"\nSản phẩm ID: {row['product_id']}")
            print(f"Tên sản phẩm: {row['product_name']}")
            print(f"Mô tả RAG: {row['rag_description']}")
            print("-" * 40)
    else:
        print("Không có dữ liệu sản phẩm để hiển thị rag_description.")
else:
    print("LỖI: Không tìm thấy DataFrame phù hợp (df_rag_input, df_final, df_merged) chứa 'rag_description'.")
    print("Hãy đảm bảo bạn đã chạy các cell tiền xử lý dữ liệu (đến Cell 5 hoặc 6) và tải lại dữ liệu (Cell 7 hoặc 9.1).")


In [ ]:
# Đảm bảo các hàm từ Cell 10 đã được định nghĩa và các biến cần thiết
# (embedding_model, faiss_index, df_rag_input, can_run_rag) đã sẵn sàng.

sample_queries_for_context_appendix = [
    "Tôi muốn mua sách về kinh tế",
    "Sách về Phật Pháp căn bản",
    "Tôi muốn mua sách về tình bạn để tặng cho bạn thân"
]
top_k_for_context_demo = 3 # Số lượng sản phẩm trong ngữ cảnh bạn muốn hiển thị

if 'rag_pipeline' in globals() and 'can_run_rag' in globals() and can_run_rag:
    for query in sample_queries_for_context_appendix:
        print(f"\n\n--- NGỮ CẢNH TRUY XUẤT CHO TRUY VẤN: '{query}' ---")
        query_embedding = get_query_embedding(query)
        if query_embedding is not None:
            distances, indices = search_faiss_index(query_embedding, top_k=top_k_for_context_demo)
            if indices.size > 0:
                # Sử dụng hàm format_retrieved_context từ Cell 10 để định dạng output
                # Hàm này đã được thiết kế để tạo context cho LLM
                retrieved_context_str = format_retrieved_context(indices, distances)
                print(retrieved_context_str)
            else:
                print("Không tìm thấy sản phẩm nào phù hợp trong FAISS index cho truy vấn này.")
        else:
            print("Không thể tạo embedding cho truy vấn.")
        print("=" * 70)
else:
    print("LỖI: Hàm rag_pipeline hoặc các thành phần RAG chưa sẵn sàng. Vui lòng chạy lại Cell 10 và đảm bảo các biến cần thiết đã được khởi tạo.")



In [ ]:
# Đảm bảo Cell 12 đã chạy và biến benchmark_df_results_final (hoặc tên bạn đặt cho kết quả benchmark) đã tồn tại.

df_benchmark_to_show = None
if 'benchmark_df_results_final' in globals() and not benchmark_df_results_final.empty:
    df_benchmark_to_show = benchmark_df_results_final
elif 'benchmark_df_results' in globals() and not benchmark_df_results.empty: # Kiểm tra tên biến cũ hơn
    df_benchmark_to_show = benchmark_df_results


if df_benchmark_to_show is not None:
    print("--- Kết quả chi tiết của một vài Test Case Benchmark ---")

    # Chọn các cột bạn muốn hiển thị trong phụ lục
    columns_for_appendix = [
        'query',
        'rag_answer',
        'rag_retrieved_ids',
        'ground_truth_doc_ids',
        'rag_context_precision',
        'rag_context_recall',
        'rag_faithfulness',
        'rag_answer_relevance',
        'pure_llm_answer', # Có thể thêm để so sánh
        'pure_llm_answer_relevance', # Có thể thêm để so sánh
        'trad_search_retrieved_ids', # Có thể thêm để so sánh
        'trad_search_precision', # Có thể thêm để so sánh
        'trad_search_recall' # Có thể thêm để so sánh
    ]

    # Lọc bỏ các cột không tồn tại trong DataFrame (nếu có)
    existing_columns_for_appendix = [col for col in columns_for_appendix if col in df_benchmark_to_show.columns]

    # Hiển thị 3-5 test case đầu tiên. Bạn có thể thay đổi số lượng hoặc chọn các hàng cụ thể.
    num_test_cases_to_show = min(5, len(df_benchmark_to_show))

    if num_test_cases_to_show > 0:
        # Sử dụng to_markdown() để có định dạng bảng đẹp hơn, hoặc to_string()
        try:
            print(df_benchmark_to_show[existing_columns_for_appendix].head(num_test_cases_to_show).to_markdown(index=False))
        except ImportError: # Nếu to_markdown không có (phiên bản pandas cũ)
            print(df_benchmark_to_show[existing_columns_for_appendix].head(num_test_cases_to_show).to_string())
    else:
        print("Không có kết quả benchmark để hiển thị.")
else:
    print("LỖI: Không tìm thấy DataFrame kết quả benchmark ('benchmark_df_results_final' hoặc 'benchmark_df_results').")
    print("Hãy đảm bảo bạn đã chạy Cell 12 thành công và không gặp lỗi API quota cho các test case bạn muốn xem.")
